In [ ]:
# imports 
import requests
import pandas as pd
from sqlalchemy import create_engine
from time import sleep 

# SQL database connection
DB_USER = "root"
DB_PASSWORD = "_Joseph344"
DB_HOST = "localhost"
DB_NAME = "NFL_Project2"

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")

def get_roster_links():
    url = "https://site.web.api.espn.com/apis/site/v2/sports/football/nfl/teams"
    response = requests.get(url, timeout=10)
    data = response.json()
    
    roster_links = []
    teams = data["sports"][0]["leagues"][0]["teams"]
    
    for team in teams:
        links = team["team"]["links"]
        for link in links:
            if "roster" in link["rel"]:
                # Convert to API endpoint format
                team_id = team["team"]["id"]
                api_url = f"https://site.web.api.espn.com/apis/site/v2/sports/football/nfl/teams/{team_id}/roster"
                roster_links.append(api_url)
                break
    return roster_links

def process_roster(url):
    try:
        response = requests.get(url, timeout=10)
        data = response.json()
        team_name = data.get("team", {}).get("displayName", "Unknown Team")
        
        offense_players = []
        defense_players = []
        special_teams = []
        
        athlete_groups = data.get("athletes", [])
        
        for group in athlete_groups:
            position_group = group.get("position", "").lower()
            players = group.get("items", [])
            
            for player in players:
                player_data = {
                    "name": player.get("fullName", "N/A"),
                    "id": player.get("id", "na"),
                    "team": team_name,
                    "position": player.get("position", {}).get("abbreviation", "na"),
                    "height": player.get("displayHeight", "na"),
                    "weight": player.get("displayWeight", "na"),
                    "age": player.get("age", "na"),
                    "college": player.get("college", {}).get("shortName", "na"),
                    "jersey": player.get("jersey", "na"),
                    "experience": player.get("experience", {}).get("years", "na")
                }
                
                if position_group == "offense":
                    offense_players.append(player_data)
                elif position_group == "defense":
                    defense_players.append(player_data)
                else:
                    special_teams.append(player_data)
        
        return offense_players, defense_players, special_teams
    
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return [], [], []

def process_all_rosters():
    roster_urls = get_roster_links()
    all_offense = []
    all_defense = []
    all_special = []
    
    for i, url in enumerate(roster_urls, 1):
        print(f"Processing {i}/{len(roster_urls)}: {url.split('/')[-2]}")
        offense, defense, special = process_roster(url)
        all_offense.extend(offense)
        all_defense.extend(defense)
        all_special.extend(special)
        sleep(0.5)  
    
    # Create DataFrames
    offense_df = pd.DataFrame(all_offense)
    defense_df = pd.DataFrame(all_defense)
    special_df = pd.DataFrame(all_special)

            # Export to MySQL
    if not offense_df.empty:
        offense_df.to_sql("offense_attributes", con=engine, if_exists="append", index=False)
        print(f"Offensive stats for Game ID inserted into MySQL ✅")
    
    if not defense_df.empty:
        defense_df.to_sql("defense_attributes", con=engine, if_exists="append", index=False)
        print(f"Offensive stats for Game ID inserted into MySQL ✅")
    

        # Save to CSV
    offense_df.to_csv("/Users/mattatchison/Documents/SQL_Projects/raw_all_player_attributes_offense.csv", index=False)
    defense_df.to_csv("/Users/mattatchison/Documents/SQL_Projects/raw_all_player_attributes_defense.csv", index=False)
    special_df.to_csv("/Users/mattatchison/Documents/SQL_Projects/raw_all_player_attributes_speacial.csv", index=False)

    
    print(f"\nCompleted processing:\n"
          f"- Offensive players: {len(offense_df)}\n"
          f"- Defensive players: {len(defense_df)}\n"
          f"- Special teams: {len(special_df)}")
    
    return offense_df, defense_df, special_df

# Run the processor
offense, defense, special = process_all_rosters()